In [ ]:
import nengo
import nengo_bio as bio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec

bio.set_defaults()

In [ ]:
with nengo.Network(seed=4581) as net:
    ens_exc = bio.Ensemble(
        n_neurons=10, dimensions=1, p_exc=1.0,
        max_rates=nengo.dists.Uniform(25, 50),
        locations=bio.NeuralSheetDist(dimensions=1))
    ens_inh = bio.Ensemble(
        n_neurons=20, dimensions=1, p_inh=1.0,
        max_rates=nengo.dists.Uniform(10, 20),
        locations=bio.NeuralSheetDist(dimensions=1))
    conn = bio.Connection(ens_exc, ens_inh,
        connectivity=bio.SpatiallyConstrainedConnectivity(
            divergence=5, sigma=0.25),)

    nd_input = nengo.Node(nengo.processes.WhiteSignal(period=10.0, high=1.0, rms=0.5))
    nengo.Connection(nd_input, ens_exc)

    p_exc_dec = nengo.Probe(ens_exc, synapse=0.1)
    p_exc_spikes = nengo.Probe(ens_exc.neurons, synapse=None)

    p_inh_dec = nengo.Probe(ens_inh, synapse=0.1)
    p_inh_spikes = nengo.Probe(ens_inh.neurons, synapse=None)

In [ ]:
with nengo.Simulator(net) as sim:
    sim.run(10.0)

In [ ]:
def plot(only_spike_raster=False):
    alpha = 0.0 if only_spike_raster else 1.0
    src_locs = sim.data[ens_exc].locations
    tar_locs = sim.data[ens_inh].locations

    fig = plt.figure(figsize=(3.45, 1.5), constrained_layout=True)
    spec = gridspec.GridSpec(ncols=2, nrows=1, figure=fig)
    #ax1 = fig.add_subplot(spec[0, 0:2])
    #ax2 = fig.add_subplot(spec[1, 0])
    #ax3 = fig.add_subplot(spec[1, 1])
    ax2 = fig.add_subplot(spec[0, 0])
    ax3 = fig.add_subplot(spec[0, 1])

    color0 = np.asarray(cm.get_cmap('tab10')(0.0))
    color1 = np.asarray(cm.get_cmap('tab10')(0.1))

    for i in range(ens_exc.n_neurons):
        for j in range(ens_inh.n_neurons):
            if sim.data[conn].connectivity[0, i, j] and np.abs(sim.data[conn].weights[bio.Excitatory][i, j]) > 1e-15:
                ax2.quiver([-1.0], [src_locs[i]], [2.0], [tar_locs[j] - src_locs[i]], scale=1.03, units='xy', linewidth=0.5, headwidth=5.0, headlength=5.0, alpha=alpha)

    ax2.set_ylabel('Location in 1D Sheet')
    ax2.set_xticks([-1.0, 1.0])
    ax2.set_xticklabels(['Pre', 'Post'])
    ax2.scatter(-np.ones(ens_exc.n_neurons), src_locs, color=color0, zorder=10, s=10.0, edgecolor='k', linewidth=0.5, alpha=alpha)
    ax2.scatter( np.ones(ens_inh.n_neurons), tar_locs, color=color1, zorder=10, marker='s', s=10.0, edgecolor='k', linewidth=0.5, alpha=alpha)
    ax2.set_aspect(1.0)

    ts = sim.trange()
    spikes_exc = sim.data[p_exc_spikes]
    spikes_inh = sim.data[p_inh_spikes]
    if only_spike_raster:
        h, y1, n = 3.0, -1.5, 30
        for i in range(ens_exc.n_neurons):
            y0 = y1
            y1 = y0 + h / (n + 1)
            for idx in np.where(spikes_exc[:, i] > 0.0)[0]:
                ax3.plot([ts[idx], ts[idx]], [y0, y1], color=np.minimum(1.0, 0.5 + 0.5 * color0), linewidth=0.25, solid_capstyle='butt')
        for i in range(ens_inh.n_neurons):
            y0 = y1
            y1 = y0 + h / (n + 1)
            for idx in np.where(spikes_inh[:, i] > 0.0)[0]:
                ax3.plot([ts[idx], ts[idx]], [y0, y1], color=np.minimum(1.0, 0.5 + 0.5 * color1), linewidth=0.25, solid_capstyle='butt')

    ax3.plot(ts, np.zeros_like(sim.trange()), 'k:', linewidth=0.5, alpha=alpha)
    ax3.plot(ts, sim.data[p_exc_dec], alpha=alpha)
    ax3.plot(ts, sim.data[p_inh_dec], alpha=alpha)
    ax3.set_ylim(-1.5, 1.5)
    ax3.set_yticks([])
    ax3.set_xticks([])
    ax3.set_aspect(3.7)
    ax3.plot([8, 10], [-0.9, -0.9], 'k', linewidth=2, solid_capstyle='butt', alpha=alpha)
    ax3.text(9, -1.0, '$2\\,\\mathrm{s}$', va='top', ha='center', alpha=alpha)

    ax3.set_xlabel('Time $t$ (s)')
    ax3.set_ylabel('Decoded value')

    for ax in [ax2, ax3]:
        for spine in ['left', 'top', 'right', 'bottom']:
            ax.spines[spine].set_visible(False)

    return fig

fig1 = plot(False)
fig1.savefig('nengo_bio_example.pdf', transparent=True)

fig2 = plot(True)
fig2.savefig('nengo_bio_example.png', dpi=300, transparent=False)
